In [1]:
import numpy as np
import cv2, os, random, time
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout
from tensorflow.keras.applications.vgg16 import VGG16
import xml.etree.ElementTree as ET


In [2]:
def get_image_bounding_box(xml_path, dataset_type=True):
    
    tree = ET.parse(xml_path)
    root = tree.getroot()
    bb = root.findall("./object/bndbox")[0]

    xmin = int(bb.find("xmin").text)
    xmax = int(bb.find("xmax").text)
    ymin = int(bb.find("ymin").text)
    ymax = int(bb.find("ymax").text)
    
    return ((xmin, ymin), (xmax, ymax)) if not(dataset_type) else list(map(float, [xmin, ymin, xmax, ymax]))

In [3]:
#Prepare dataset
X, y = [], []
arch_path = "archive"
for i in os.listdir(arch_path + "/images"):
    img = cv2.imread(arch_path + "/images/" + i)
    fname = i.replace(".jpg", "")
    X.append(img)
    y.append(get_image_bounding_box(arch_path + "/annotations/" + fname + ".xml"))
X = np.array(X)
y = np.array(y)

In [ ]:
del X, y

In [10]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=random.randint(0, 100))

In [6]:
#model
model = Sequential(layers=VGG16(input_shape = (480, 640, 3), 
    include_top = False,
    weights = 'imagenet').layers)
for i in model.layers:
    if "convolution" in str(i):
        i.activation = keras.activations.relu
        print(i)
model.add(Flatten())
#model.add(Dense(100, activation="relu"))
model.add(Dense(50, activation="relu"))
model.add(Dense(25, activation="relu"))
model.add(Dense(4, activation="relu"))


In [5]:
#load model
model = keras.models.load_model("smoke_detection.h5")

In [11]:
model.compile(loss="mae", optimizer=keras.optimizers.Adam(epsilon=1.0, lr=0.00001), metrics=["acc"])

In [12]:
history = model.fit(train_x, train_y, batch_size=2, epochs=2, validation_split=0.1)

Epoch 1/2
265/265 [==============================] - 158s 593ms/step - loss: 14.2674 - acc: 0.9792 - val_loss: 12.5420 - val_acc: 1.0000
Epoch 2/2
265/265 [==============================] - 171s 644ms/step - loss: 14.1471 - acc: 0.9792 - val_loss: 12.5717 - val_acc: 1.0000


In [14]:
#save model
model.save("smoke_detection.h5")

In [13]:
#evaluate model
model.evaluate(test_x, test_y, batch_size=2)

74/74 [==============================] - 13s 177ms/step - loss: 13.7671 - acc: 1.0000


[13.767096519470215, 1.0]